In [10]:
from dotenv import load_dotenv
import os

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, FunctionTool
from google.genai import types

In [11]:
load_dotenv()

True

In [12]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

### Parallel + Sequential Design Patterns

In [18]:
tech_researcher = Agent(
    name="TechResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
        Research the latest AI/ML trends. Include 3 key developments, the main 
        companies involved, and the potential impact. Keep the report very 
        concise (100 words).
    """,
    tools=[google_search],
    output_key="tech_research"
)

In [21]:
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
        Research recent medical breakthroughs. Include 3 significant advances, 
        their practical applications, and estimated timelines. Keep the report 
        concise (100 words).
    """,
    tools=[google_search],
    output_key="health_research"
)


In [22]:
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
        Research recent medical breakthroughs. Include 3 significant advances,
        their practical applications, and estimated timelines. Keep the report 
        concise (100 words).
    """,
    tools=[google_search],
    output_key="finance_research"
)


In [23]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
        Combine these three research findings into a single executive summary:

        **Technology Trends:**
        {tech_research}
    
        **Health Breakthroughs:**
        {health_research}
    
        **Finance Innovations:**
        {finance_research}
    
        Your summary should highlight common themes, surprising connections, and 
        the most important key takeaways from all three reports. The final summary 
        should be around 200 words.
    """,
    output_key="executive_summary",  # This will be the final output of the entire system.
)


In [24]:
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

In [25]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
FinanceResearcher > ## Daily Executive Briefing: Tech, Health, and Finance

**Technology:** AI democratization is rapidly advancing, making sophisticated tools accessible to more users via open-source models and cloud APIs. This trend is expected to continue its high impact throughout 2025 and beyond, fostering innovation and faster product launches. The expansion of the Internet of Things (IoT) is also a major focus, with an increasing number of connected devices transforming smart services.

**Health:** Gene therapy, initially trialed in 1990, is now a recognized treatment for certain genetic diseases and blindness, with ongoing research for conditions like heart disease and cystic fibrosis. Advancements in AI are also impacting healthcare, with projections suggesting AI in financial services will double by 2029, implying similar growth in medical applications for diagnos

### Loop Design Pattern

In [27]:
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
        Based on the user's prompt, write the first draft of a short story 
        (around 100-150 words). Output only the story text, with no introduction 
        or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

In [29]:
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
        You are a constructive story critic. Review the story provided below.

        Story: {current_story}
    
         Evaluate the story's plot, characters, and pacing.
         - If the story is well-written and complete, you MUST respond with 
        the exact phrase: "APPROVED"
         - Otherwise, provide 2-3 specific, actionable suggestions for 
         improvement.
    """,
    output_key="critique",  # Stores the feedback in the state.
)

In [30]:
def exit_loop():
    """
    Call this function ONLY when the critique is 'APPROVED', indicating the 
    story is finished and no more changes are needed.
    """
    
    return {
        "status": "approved", "message": "Story approved. Exiting refinement loop."
    }


In [ ]:
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function 
    and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback 
    from the critique.
    """,
    output_key="current_story",  
    tools=[
        FunctionTool(exit_loop)
    ],  
)


In [32]:
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2, 
)

root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)


In [33]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias traced the worn grooves of the lighthouse wall, the salt-laced air a constant companion. For twenty years, the beam had been his life, a silent promise to the churning sea. Tonight, however, a flicker from the abandoned storage room caught his eye. He pushed open the creaking door, dust motes dancing in the single shaft of moonlight. There, on a forgotten sea chest, lay a map. It wasn't parchment, but something smoother, cooler to the touch. And it glowed. Faint, ethereal lines pulsed, charting unseen islands and currents that defied any known cartography. Elias, a man of routine, felt a thrill, sharp and unexpected, pierce the quiet solitude of his life. The sea, it seemed, still held its secrets.
CriticAgent > This is a promising start to a mystery! The atmosphere is well-established, and the discovery of the glowing map 